In [ ]:
from google.colab import drive
drive.mount('/drive')

from PIL import Image
from numpy import array
import pandas as pd
import numpy as np
import PIL
import matplotlib
import matplotlib.pyplot as plt

approx_displacement = 40 
image_left = Image.open(r"/drive/My Drive/Colab Notebooks/mlsp1/im0.ppm")
left_image_array = np.array(image_left)
image_right = Image.open(r"/drive/My Drive/Colab Notebooks/mlsp1/im8.ppm")
right_image_array = np.array(image_right)
min_displacement_factor = 9999
shape = (left_image_array.shape[0],left_image_array.shape[1]-approx_displacement)
disparity_matrix = np.ones(shape)

Disparity matrix : Which pixel in the left image resembles the picture in the right_source image the most. Record the lateral displacement of this image

In [ ]:
for row in range(left_image_array.shape[0]):
    for col in range(left_image_array.shape[1]-approx_displacement):
        min_displacement_factor = 9999
        for search_radii in range(approx_displacement):
            displacement_factor = np.sum(np.abs(left_image_array[row,col+search_radii] - right_image_array[row,col]))
            if displacement_factor < min_displacement_factor:
                #print('displacement_factor/cost',displacement_factor)
                #print('k',search_radii)
                min_displacement_factor = displacement_factor
                disparity_matrix[row,col] = search_radii 
                #print(disparity_matrix)
disparity_matrix

Plotting that disparity matrix and histogram

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
colour_graph = ax.matshow(disparity_matrix)
fig.colorbar(colour_graph)
plt.show()
plt.hist(disparity_matrix.reshape(-1),approx_displacement)
plt.show()

A self defined clustering algorithm to cluster the pixels into the 4 clusters

In [ ]:
import random
import statistics
Centroids = np.array(4)
Centroids = [1,10,20,30]
# randomly appointed centroids 
initial_centroid = np.zeros(4)

#print(Centroids)

Cen0collection = []
Cen1collection = []
Cen2collection=[]
Cen3collection =[]

disparity_copy  = disparity_matrix.reshape(-1)
cluster_no_holder = np.ones(disparity_copy.shape)
itr = 50
while(itr>0):
      #print(itr)
      # finding to which cluster it belongs
      for i in range(disparity_copy.shape[0]):
        Min_Diff = 40
        for k in range(0,4):
            Diff = abs(Centroids[k] - disparity_copy[i])
            if Diff < Min_Diff:
              Min_Diff = Diff
              #print(i,Diff,disparity_copy[i])
              cluster_no_holder[i] = k

       # placing the cluster number  
      for i in range(disparity_copy.shape[0]):
        if cluster_no_holder[i] == 0:
          Cen0collection.append(disparity_copy[i])
        if cluster_no_holder[i] == 1:
          Cen1collection.append(disparity_copy[i])
        if cluster_no_holder[i] == 2:
          Cen2collection.append(disparity_copy[i])
        if cluster_no_holder[i] == 3:
          Cen3collection.append(disparity_copy[i])


      # recalculating the cluster centroids
      Centroids[0] = statistics.mean(Cen0collection)
      Centroids[1] = statistics.mean(Cen1collection)
      Centroids[2] = statistics.mean(Cen2collection)
      Centroids[3] = statistics.mean(Cen3collection)

      #print(Centroids[0],Centroids[1],Centroids[2],Centroids[3])

      if (Centroids[0] - initial_centroid[0]) + (Centroids[1] - initial_centroid[1]) + (Centroids[2] - initial_centroid[2]) == 0:
        break
      else:
        initial_centroid[0] = Centroids[0]
        initial_centroid[1] = Centroids[1]
        initial_centroid[2] = Centroids[2]
        initial_centroid[3] = Centroids[3]

      itr-=1

Printing values of clusters

In [ ]:
print(Centroids)

disparity_matrix_new = disparity_matrix

for row in range(left_image_array.shape[0]):
    for col in range(left_image_array.shape[1]-approx_displacement):
      diff = 40
      for i in range(0,4):
        if(abs(disparity_matrix_new[row,col]-Centroids[i]) < diff):
          centroid_number = i
          diff = abs(disparity_matrix[row,col]-Centroids[i])
        
      disparity_matrix_new[row,col] = Centroids[centroid_number]

Plotting images again 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
colour_graph = ax.matshow(disparity_matrix_new)
fig.colorbar(colour_graph)
plt.show()